In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam


In [ ]:
df = pd.read_csv("creditcard.csv")

# Use only "normal" (non-fraud) data for training
normal_data = df[df['Class'] == 0]
fraud_data  = df[df['Class'] == 1]